In [1]:
# 📦 Imports nécessaires
import pandas as pd

In [12]:
# 📂 Chargement des données
df = pd.read_csv("output/jobs_infos.csv")

In [13]:
# 🧼 Nettoyage basique des TJM
df["TJM_min"] = df["TJM"].str.extract(r'(\d+)\s*€')
df["TJM_max"] = df["TJM"].str.extract(r'(\d+)[^0-9]+(\d+)').iloc[:,1]
df["TJM_min"] = pd.to_numeric(df["TJM_min"], errors="coerce")
df["TJM_max"] = pd.to_numeric(df["TJM_max"], errors="coerce")
df["TJM_moyen"] = df[["TJM_min", "TJM_max"]].mean(axis=1)

# 🧼 Nettoyage des durées
df["DURATION_clean"] = df["DURATION"].str.extract(r'(\d+)').astype(float)

# 📊 Statistiques générales
print("📌 Nombre total d'offres :", len(df))
print("\n📌 TJM moyen (hors NaN) :", round(df["TJM_moyen"].mean(), 2), "€")

# 📌 Durée moyenne (en mois)
print("\n📌 Durée moyenne (en mois) :", round(df["DURATION_clean"].mean(), 2))

# 📌 Répartition par expérience
print("\n📌 Répartition des niveaux d'expérience :")
print(df["EXPERIENCE"].value_counts())

# 📌 Répartition géographique
print("\n📌 Répartition géographique :")
print(df["LOCATION"].value_counts())

# 📌 Aperçu des offres triées par TJM
print("\n📌 Top offres par TJM moyen :")
print(df[["TITLE", "TJM_moyen"]].sort_values(by="TJM_moyen", ascending=False))

# 📌 Missions sans TJM ou durée renseignée
print("\n📌 Offres sans TJM ou durée renseignée :")
print(df[df["TJM_moyen"].isna() | df["DURATION_clean"].isna()][["TITLE", "TJM", "DURATION"]])

📌 Nombre total d'offres : 64

📌 TJM moyen (hors NaN) : 468.71 €

📌 Durée moyenne (en mois) : 9.92

📌 Répartition des niveaux d'expérience :
EXPERIENCE
5 à 10 ans d’expérience    43
2 à 5 ans d’expérience     13
> 10 ans d’expérience       7
< 2 ans d’expérience        1
Name: count, dtype: int64

📌 Répartition géographique :
LOCATION
Paris, France                                  17
Paris, Île-de-France                            6
Saint-Herblain, Pays de la Loire                5
France                                          4
Lille, Hauts-de-France                          3
Toulouse, Occitanie                             3
Lyon, Auvergne-Rhône-Alpes                      3
Montpellier, Occitanie                          2
Provence-Alpes-Côte d'Azur, France              1
Bagneux, Île-de-France                          1
Gémenos, Provence-Alpes-Côte d'Azur             1
Île-de-France, France                           1
Grand Est, France                               1
Aix-en-Provenc

In [30]:
# 🔍 Définir le skill à chercher
skill = "python"

# 🧼 Nettoyer un peu le texte : on peut tout passer en minuscule pour éviter les problèmes de casse
df["SKILLS_str"] = df["SKILLS"].fillna("").str.lower()

# 🔍 Filtrer les lignes contenant le mot-clé (skill)
filtered = df[df["SKILLS_str"].str.contains(skill.lower())]

# 🖨️ Résultat
print(f"\n🎯 Offres contenant le skill '{skill}':")
print(filtered[["TITLE", "SKILLS", "LOCATION"]])


🎯 Offres contenant le skill 'python':
                                              TITLE  \
4   Développeur Rust - Full remote - Anglais fluent   
21      Data Engineer Dev Palantir Skywise & Python   
55                                 Ingénieur DevOps   

                                               SKILLS  \
4                                  ['Python', 'Rust']   
21                                         ['Python']   
55  ['Agile Scrum', 'Apache Kafka', 'Bash', 'BigQu...   

                                 LOCATION  
4                           Paris, France  
21  Marignane, Provence-Alpes-Côte d'Azur  
55                                  Maroc  


In [21]:
from collections import Counter
import re

# 🧼 Convertir tous les skills en minuscules, nettoyer les crochets et guillemets
skills_series = df["SKILLS"].dropna().str.lower()

# 🔍 Extraire tous les mots entre quotes avec regex
all_skills = []

for skills_str in skills_series:
    # On extrait tous les mots entre quotes simples : 'python', 'rust'...
    skills = re.findall(r"'([^']+)'", skills_str)
    all_skills.extend(skills)

# 🧮 Compter les occurrences
skill_counts = Counter(all_skills)

# 🔝 Afficher le top 10
print("💡 Compétences les plus demandées :")
for skill, count in skill_counts.most_common(10):
    print(f"{skill}: {count} fois")

💡 Compétences les plus demandées :
sap: 8 fois
java: 4 fois
python: 3 fois
cybersécurité: 3 fois
.net: 3 fois
aws cloud: 2 fois
iam: 2 fois
pentesting: 2 fois
devops: 2 fois
endpoint detection and response (edr): 2 fois
